<a href="https://colab.research.google.com/github/bkkaggle/pytorch-CycleGAN-and-pix2pix/blob/master/pix2pix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install

In [ ]:
# !git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

In [4]:
import os
os.chdir(r'C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix/')

In [3]:
!pip install -r requirements.txt

# Datasets

Download one of the official datasets with:

-   `bash ./datasets/download_pix2pix_dataset.sh [cityscapes, night2day, edges2handbags, edges2shoes, facades, maps]`

Or use your own dataset by creating the appropriate folders and adding in the images. Follow the instructions [here](https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix/blob/master/docs/datasets.md#pix2pix-datasets).

### Dataset creation for HE (B):

In [1]:
import os
import random
import shutil

# Set the seed for random shuffle
random.seed(42) # repeatability


path = r'\\shelter\Kyu\unstain2stain\unstain2stain_tile\pix2pix\train\HE\all'
train_path = r'C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets\B\train'
valid_path = r'C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets\B\val'
test_path = r'C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets\B\test'

if not os.path.exists(train_path):
    os.makedirs(train_path)

if not os.path.exists(valid_path):
    os.makedirs(valid_path)

if not os.path.exists(test_path):
    os.makedirs(test_path)

# Set the percentage split for train, validation, and test
train_pct = 0.8
valid_pct = 0.1
test_pct = 0.1

image_files = [f for f in os.listdir(path) if f.endswith('.png')]

random.shuffle(image_files)

num_files = len(image_files)
num_train = int(num_files * train_pct)
num_valid = int(num_files * valid_pct)
num_test = int(num_files * test_pct)

# Copy the image files into the train folder
for i in range(num_train):
    src_path = os.path.join(path, image_files[i])
    dst_path = os.path.join(train_path, image_files[i])
    shutil.copyfile(src_path, dst_path)

# Copy the image files into the validation folder
for i in range(num_train, num_train+num_valid):
    src_path = os.path.join(path, image_files[i])
    dst_path = os.path.join(valid_path, image_files[i])
    shutil.copyfile(src_path, dst_path)

# Copy the image files into the test folder
for i in range(num_train+num_valid, num_files):
    src_path = os.path.join(path, image_files[i])
    dst_path = os.path.join(test_path, image_files[i])
    shutil.copyfile(src_path, dst_path)


### Dataset creation for Unstained (A):

In [5]:
import os
import shutil

# Set up the paths for the original directories and the new images directory
parent_dir = r'C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets\B'
new_images_dir = r'\\shelter\Kyu\unstain2stain\unstain2stain_tile\pix2pix\train\Unstained\all'
new_save_dir = r'C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets\A'

# Loop through each original directory (train, val, test)
for split_dir in os.listdir(parent_dir):
    split_path = os.path.join(parent_dir, split_dir)

    # Get the set of image file names for this split
    split_images = set(os.listdir(split_path))

    # Loop through each image in the new images directory
    for image_file in os.listdir(new_images_dir):

        # Check if the image is in the current split
        if image_file in split_images:
            image_path = os.path.join(new_images_dir, image_file)
            new_split_path = os.path.join(new_save_dir, split_dir, image_file)
            shutil.copyfile(image_path, new_split_path)


In [1]:
import os

# Set up the paths for directories A and B
dir_a = r'C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets\A'
dir_b = r'C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets\B'

# Loop through each split (train, val, test)
for split in ['train', 'val', 'test']:
    split_a_path = os.path.join(dir_a, split)
    split_b_path = os.path.join(dir_b, split)

    # Get the set of image file names for each split in directory A
    split_a_images = set(os.listdir(split_a_path))

    # Get the set of image file names for each split in directory B
    split_b_images = set(os.listdir(split_b_path))

    # Find the set of images that are in directory A but not in directory B
    only_in_a = split_a_images - split_b_images

    # Find the set of images that are in directory B but not in directory A
    only_in_b = split_b_images - split_a_images

    # Print the results
    print(f"Split: {split}")
    print(f"Only in {dir_a}: {len(only_in_a)}")
    print(f"Only in {dir_b}: {len(only_in_b)}")
    print("-----------")


Split: train
Only in C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets\A: 0
Only in C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets\B: 0
-----------
Split: val
Only in C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets\A: 0
Only in C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets\B: 0
-----------
Split: test
Only in C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets\A: 0
Only in C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets\B: 0
-----------


In [1]:
# !bash ./datasets/download_pix2pix_dataset.sh facades

Split: train
Only in C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets\A: 0
Only in C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets\B: 0
-----------
Split: val
Only in C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets\A: 0
Only in C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets\B: 0
-----------
Split: test
Only in C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets\A: 0
Only in C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets\B: 0
-----------


## Run this to finish generating dataset:

In [3]:
!python datasets/combine_A_and_B.py --fold_A C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets\A --fold_B C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets\B --fold_AB C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets --no_multiprocessing

[fold_A] =  C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets\A
[fold_B] =  C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets\B
[fold_AB] =  C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets
[num_imgs] =  1000000
[use_AB] =  False
[no_multiprocessing] =  True
split = test, use 357/357 images
split = test, number of images = 357
split = train, use 2856/2856 images
split = train, number of images = 2856
split = val, use 357/357 images
split = val, number of images = 357


# Pretrained models

Download one of the official pretrained models with:

-   `bash ./scripts/download_pix2pix_model.sh [edges2shoes, sat2map, map2sat, facades_label2photo, and day2night]`

Or add your own pretrained model to `./checkpoints/{NAME}_pretrained/latest_net_G.pt`

In [ ]:
# !bash ./scripts/download_pix2pix_model.sh facades_label2photo

In [4]:
import torch
torch.cuda.is_available()

True

# Training

-   `python train.py --dataroot ./datasets/facades --name facades_pix2pix --model pix2pix --direction BtoA`

Change the `--dataroot` and `--name` to your own dataset's path and model's name. Use `--gpu_ids 0,1,..` to train on multiple GPUs and `--batch_size` to change the batch size. Add `--direction BtoA` if you want to train a model to transform from class B to A.

In [1]:
!python train.py --dataroot C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets --name unstain2stain_pix2pix --model pix2pix --direction AtoB --display_id -1 --display_freq 400 --display_ncols 4 --update_html_freq 1000 --print_freq 100 --checkpoints_dir C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\checkpoints --save_epoch_freq 5 --n_epochs 1000

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

C:\Users\Kevin\.conda\envs\wsi_analysis1\lib\site-packages\torch\optim\lr_scheduler.py:138: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


# Testing

-   `python test.py --dataroot ./datasets/facades --direction BtoA --model pix2pix --name facades_pix2pix`

Change the `--dataroot`, `--name`, and `--direction` to be consistent with your trained model's configuration and how you want to transform images.

> from https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix:
> Note that we specified --direction BtoA as Facades dataset's A to B direction is photos to labels.

> If you would like to apply a pre-trained model to a collection of input images (rather than image pairs), please use --model test option. See ./scripts/test_single.sh for how to apply a model to Facade label maps (stored in the directory facades/testB).

> See a list of currently available models at ./scripts/download_pix2pix_model.sh

In [ ]:
# !ls checkpoints/

In [2]:
!python test.py --dataroot C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets --direction AtoB --model pix2pix --name unstain2stain_pix2pix --results_dir C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\results --num_test 50

----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 256                           
                 dataroot: C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets	[default: None]
             dataset_mode: aligned                       
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
                  isTrain: False                         	[default: None]
          

### Inference for proof-of-concept: Tiled original unstained image -> run inference -> Output stained image tiles -> Reconstruct to fake WSI HE Image

In [ ]:
!python test.py --dataroot C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\datasets --direction AtoB --model pix2pix --name poc_pix2pix --results_dir C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\results # --num_test 5000

# Visualize

In [ ]:
import matplotlib.pyplot as plt

img = plt.imread('./results/facades_label2photo_pretrained/test_latest/images/100_fake_B.png')
plt.imshow(img)

In [ ]:
img = plt.imread('./results/facades_label2photo_pretrained/test_latest/images/100_real_A.png')
plt.imshow(img)

In [ ]:
img = plt.imread('./results/facades_label2photo_pretrained/test_latest/images/100_real_B.png')
plt.imshow(img)